In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("app1").getOrCreate()
spark

In [12]:

df = spark.read.format("csv")\
    .option("inferSchema","true") \
    .option("header", "true") \
        .load("./data/qs-world-university-rankings-2017-to-2022-V2.csv")
#df.show()
df.createOrReplaceTempView("university_ranking")
df.printSchema()

root
 |-- university: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- rank_display: string (nullable = true)
 |-- score: double (nullable = true)
 |-- link: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- logo: string (nullable = true)
 |-- type: string (nullable = true)
 |-- research_output: string (nullable = true)
 |-- student_faculty_ratio: integer (nullable = true)
 |-- international_students: string (nullable = true)
 |-- size: string (nullable = true)
 |-- faculty_count: string (nullable = true)



In [13]:
spark.sql("Select university,year,rank_display,region, country from university_ranking").show(5)

+--------------------+----+------------+-------------+--------------+
|          university|year|rank_display|       region|       country|
+--------------------+----+------------+-------------+--------------+
|Massachusetts Ins...|2017|           1|North America| United States|
| Stanford University|2017|           2|North America| United States|
|  Harvard University|2017|           3|North America| United States|
|University of Cam...|2017|           4|       Europe|United Kingdom|
|California Instit...|2017|           5|North America| United States|
+--------------------+----+------------+-------------+--------------+
only showing top 5 rows



In [16]:
spark.sql("""
    select university , year,rank_display,int(rank_display) as n_rank, region, country
    from university_ranking
""").show()

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
|University of Oxford|2017|           6|     6|       Europe|United Kingdom|
|                 UCL|2017|           7|     7|       Europe|United Kingdom|
|ETH Zurich - Swis...|2017|           8|     8|       Europe|   Switzerland|
|Imperial College ...|2017|           9|     9|       Europe|United Kingdom|
|University of Chi...|2017|          10|    10|North America| United States|

In [25]:

spark.sql(""" 
   select university, year
          ,rank_display
          ,int(split(rank_display, '-')[0]) as n_rank
          ,region,country
          from university_ranking 
          where rank_display like '%-%'
""").show()

+--------------------+----+------------+------+-------------+--------------------+
|          university|year|rank_display|n_rank|       region|             country|
+--------------------+----+------------+------+-------------+--------------------+
|Bandung Institute...|2017|     401-410|   401|         Asia|           Indonesia|
| Brandeis University|2017|     401-410|   401|North America|       United States|
|Illinois Institut...|2017|     401-410|   401|North America|       United States|
|Johannes Gutenber...|2017|     401-410|   401|       Europe|             Germany|
|Khalifa Universit...|2017|     401-410|   401|         Asia|United Arab Emirates|
|National Research...|2017|     401-410|   401|       Europe|              Russia|
|Taipei Medical Un...|2017|     401-410|   401|         Asia|              Taiwan|
|Universidad Exter...|2017|     401-410|   401|Latin America|            Colombia|
|Wake Forest Unive...|2017|     401-410|   401|North America|       United States|
|Aix

In [35]:

clean_rankings_df = spark.sql("""select 
           university
          ,coalesce(int(year), 9999) as year 
          ,rank_display
          ,coalesce(int(split(rank_display, '-')[0]), 9999) as n_rank
          ,coalesce(float(score), -1) as score 
          ,region,country,city,type 
          ,coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio
          ,coalesce(int(regexp_replace(international_students, '[.,]','')),-1)  as international_students
          ,size
          ,coalesce(int(regexp_replace(faculty_count, '[.,]','')),-1)  as faculty_count
          from university_ranking 
          where rank_display like '%-%'
""")

In [33]:
spark.sql("""select 
          
    from university_ranking
""").show(6000)

+----------------------+
|international_students|
+----------------------+
|                  3730|
|                  3879|
|                  5877|
|                  7925|
|                   692|
|                  8442|
|                 21824|
|                  7733|
|                 11143|
|                  4696|
|                  1912|
|                  7551|
|                  6091|
|                  6426|
|                  2537|
|                  5675|
|                  5070|
|                  4636|
|                 14637|
|                 11252|
|                 15075|
|                  6496|
|                  7341|
|                  5420|
|                  3445|
|                  3846|
|                  8311|
|                  9428|
|                 16079|
|                 10827|
|                  6140|
|                 27536|
|                   374|
|                  3983|
|                  1914|
|                  3033|
|                  2863|


In [ ]:
clean_dynamic_frame = DynamicFrame.fromDF(clean_rankings_df, glueContext, "university_ranking_clean")

### 
university_ranking_dest = glueContext.write_dynamic_frame.from_options(
   frame =clean_dynamic_frame,
   connection_type = 's3',
   format = "csv"
   connection_options = {
       "path": "s3://aws-glue-spark/unviersity_ranking/",
       "partitionKeys": []
   },
   transformation_ctx="university_ranking_dest"    
)